### Import 

In [1]:
import pandas as pd
from sklearn.preprocessing import Normalizer, PowerTransformer, StandardScaler
from sklearn.decomposition import PCA
import matplotlib.pyplot as plt
from sklearn import linear_model
import numpy as np
from sklearn.model_selection import cross_val_predict
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.cross_decomposition import PLSRegression
from sklearn.neural_network import MLPRegressor
plt.rcParams['figure.figsize'] = [12, 8]
from sklearn import datasets, ensemble
from sklearn.inspection import permutation_importance
from sklearn.metrics import mean_squared_error
from xgboost import XGBRegressor
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import RepeatedKFold
from bokeh.plotting import figure
from bokeh.models import Select
from bokeh.io import output_notebook, show, push_notebook
from bokeh.models import ColumnDataSource, Slider, CustomJS, Div
from sklearn.model_selection import GridSearchCV
from bokeh.layouts import column, row
output_notebook()
import bokeh
import seaborn as sns
from ipywidgets import interact
from scipy.signal import savgol_filter
from sklearn.metrics import mean_absolute_error
from skopt import BayesSearchCV

Loading BokehJS ...

### Baseline correction  
https://github.com/zmzhang/airPLS

In [2]:
import numpy as np
from scipy.sparse import csc_matrix, eye, diags
from scipy.sparse.linalg import spsolve

def WhittakerSmooth(x,w,lambda_,differences=1):
    '''
    Penalized least squares algorithm for background fitting
    
    input
        x: input data (i.e. chromatogram of spectrum)
        w: binary masks (value of the mask is zero if a point belongs to peaks and one otherwise)
        lambda_: parameter that can be adjusted by user. The larger lambda is,  the smoother the resulting background
        differences: integer indicating the order of the difference of penalties
    
    output
        the fitted background vector
    '''
    X=np.matrix(x)
    m=X.size
    E=eye(m,format='csc')
    for i in range(differences):
        E=E[1:]-E[:-1] # numpy.diff() does not work with sparse matrix. This is a workaround.
    W=diags(w,0,shape=(m,m))
    A=csc_matrix(W+(lambda_*E.T*E))
    B=csc_matrix(W*X.T)
    background=spsolve(A,B)
    return np.array(background)

def airPLS(x, lambda_=100, porder=1, itermax=15):
    '''
    Adaptive iteratively reweighted penalized least squares for baseline fitting
    
    input
        x: input data (i.e. chromatogram of spectrum)
        lambda_: parameter that can be adjusted by user. The larger lambda is,  the smoother the resulting background, z
        porder: adaptive iteratively reweighted penalized least squares for baseline fitting
    
    output
        the fitted background vector
    '''
    m=x.shape[0]
    w=np.ones(m)
    for i in range(1,itermax+1):
        z=WhittakerSmooth(x,w,lambda_, porder)
        d=x-z
        dssn=np.abs(d[d<0].sum())
        if(dssn<0.001*(abs(x)).sum() or i==itermax):
            if(i==itermax): print('WARING max iteration reached!')
            break
        w[d>=0]=0 # d>0 means that this point is part of a peak, so its weight is set to 0 in order to ignore it
        w[d<0]=np.exp(i*np.abs(d[d<0])/dssn)
        w[0]=np.exp(i*(d[d<0]).max()/dssn) 
        w[-1]=w[0]
    return z


### Dataset 

https://figshare.com/projects/LIBS2022_Quantification_Contest/142250  
https://libs2022.com/

### Read data for training

In [3]:
data_training = pd.read_csv('data/train_dataset.csv')

In [4]:
data_training

,200,200.020004272461,200.039993286133,200.059997558594,200.080001831055,200.100006103516,200.119995117188,200.139999389648,200.160003662109,200.179992675781,...,999.919982910156,999.940002441406,999.960021972656,999.97998046875,1000,1000.02001953125,Cr,Mn,Mo,Ni
0,0.000010,0.001747,0.001477,0.002694,0.001689,0.001828,0.000127,-0.000101,-0.000023,0.002150,...,0.003621,0.003494,0.003204,0.003204,0.002824,-0.000054,0.045,1.960,0.0403,22.5800
1,0.002484,0.001943,0.001610,0.002062,-0.000746,0.001947,0.002961,0.003321,-0.000938,0.000817,...,0.003975,0.003234,0.001542,0.001542,0.001598,-0.000065,0.045,1.960,0.0403,22.5800
2,0.000230,0.000475,0.000754,0.001994,0.000771,0.001478,0.001255,0.001335,0.001972,0.001638,...,0.002733,0.002162,0.000860,0.000860,0.001336,-0.000043,0.045,1.960,0.0403,22.5800
3,0.001029,0.000596,0.001769,0.000886,0.000491,0.002019,0.000893,-0.001186,0.001689,0.002193,...,0.003059,0.002907,0.002559,0.002559,0.002987,-0.000055,0.045,1.960,0.0403,22.5800
4,-0.003513,0.001748,0.002183,0.000505,-0.000304,0.001544,-0.000112,0.000778,0.001538,0.000532,...,0.003133,0.003745,0.005143,0.005143,0.004133,-0.000057,0.045,1.960,0.0403,22.5800
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2095,0.002264,0.002402,-0.000559,0.002850,-0.000560,0.000547,-0.000071,0.000628,0.000101,0.001593,...,0.002244,0.002513,0.003128,0.003128,0.003266,-0.000058,4.510,0.303,3.6000,0.0309
2096,0.008210,0.000836,0.000103,0.000765,0.000508,0.005778,0.001087,0.000222,0.008658,0.001648,...,0.003443,0.002637,0.000800,0.000800,0.002035,-0.000156,4.510,0.303,3.6000,0.0309
2097,0.000226,0.006540,0.000969,-0.000720,0.002583,0.004462,0.004571,0.006759,0.003972,-0.000213,...,0.004118,0.003721,0.002817,0.002817,0.002840,-0.000084,4.510,0.303,3.6000,0.0309
2098,0.000237,0.003592,0.000563,0.000284,0.000141,0.000787,0.001087,0.002731,0.000774,-0.000290,...,0.003966,0.003397,0.002099,0.002099,0.002376,-0.000057,4.510,0.303,3.6000,0.0309


### Group by samples

In [ ]:
composition_data = data_training.groupby(['Cr','Mn','Mo','Ni'], as_index=False).count().filter(['Cr', 'Mn', 'Mo','Ni'])
composition_data['Fe'] = 100-composition_data.sum(axis=1)
composition_data['sample'] = range(len(composition_data))
composition_data 

In [ ]:
data_training_merged = pd.merge(data_training, composition_data, on = ["Cr","Mn","Mo","Ni"], how = "inner")
data_training_merged

### Divide into dependent and independent variables

In [5]:
X_training = data_training.drop(['Cr','Mn','Mo','Ni'], axis = 1)

y_training_Cr = data_training['Cr']
y_training_Mn = data_training['Mn']
y_training_Mo = data_training['Mo']
y_training_Ni = data_training['Ni']

### Plot spectra

In [ ]:
def plot_spectra(X, samples, composition_data):
    fig_spectra = figure(toolbar_location="above", x_axis_label=r"$$ \lambda [nm] $$", y_axis_label="Intensity",
                      y_range=(-0.1, 1), x_range=(200, 1000),
                      title="Spectra",width=900, height=700)

    x = X.columns
    y = X.iloc[0,:]
    line_render_data = fig_spectra.line(x, y)


    def update_plot(N):
        signal = X.iloc[int(N),:]
        line_render_data.data_source.data['y'] = signal
        composition = composition_data[composition_data['sample']==samples[int(N)]]
        fig_spectra.title.text = "Sample: "+str(samples[int(N)]) + ", Composition: Cr " + str(composition['Cr'].values[0]) + " Mn "+str(composition['Mn'].values[0]) + " Mo " + str(composition['Mo'].values[0])+ " Ni " + str(composition['Ni'].values[0])
        push_notebook()

    show(column(fig_spectra), notebook_handle=True);
    interact(update_plot, N = (0,X.shape[0]-1));

In [ ]:
plot_spectra(X = X_training, samples = data_training_merged['sample'], composition_data = composition_data)

### Filter noise

In [6]:
def filter_noise(X, treshold):
    X_limited = X.iloc[:,X.columns.astype('float')<220]
    X = X.assign(noise_level = X_limited.apply(max, axis=1))
    return X[X['noise_level'] < treshold].drop(['noise_level'],axis=1)
    

In [ ]:
X_filtered = filter_noise(X = X_training, treshold = 0.05)

In [ ]:
plot_spectra(X = X_filtered, samples = data_training_merged['sample'], composition_data = composition_data)

### Baseline corrrection

In [7]:
def correct_baseline(X):
    return X.apply(lambda row : row-airPLS(row), axis = 1)

In [ ]:
X_baseline_corrected = correct_baseline(X_filtered)

In [ ]:
plot_spectra(X = X_baseline_corrected, samples = data_training_merged['sample'], composition_data = composition_data)

### Average 3 columns

In [8]:
def average_columns(X, number_of_columns = 3):
    X_avg = X.groupby(by=np.arange(X.shape[1])//3, axis=1).mean()
    X_avg.columns = [np.mean(val) for val in X.columns.astype('float').groupby(np.arange(X.shape[1])//3).values()]
    return X_avg

In [ ]:
X_avg = average_columns(X_filtered)

In [ ]:
plot_spectra(X = X_avg,  samples = data_training_merged['sample'], composition_data = composition_data)

### Limit wavelength

In [ ]:
def limit_wavelength(X,w_min = 225, w_max = 940):
    X_limited = X.loc[:,(X.columns.astype('float')>w_min) & (X.columns.astype('float')<w_max)]
    return X_limited

### Filter lines with highest variance  
amount - number of lines to be filtered

In [ ]:
def select_wavelengts_with_big_variance(X, amount):
    return X.reindex(sorted(X[X.var().nlargest(amount).index]), axis=1)

### Update dependent variable

In [9]:
def update_dependent_variable(X, data_training):
    y_Cr = data_training[data_training.index.isin(X.index)]['Cr']
    y_Mn = data_training[data_training.index.isin(X.index)]['Mn']
    y_Mo = data_training[data_training.index.isin(X.index)]['Mo']
    y_Ni = data_training[data_training.index.isin(X.index)]['Ni']
    return y_Cr, y_Mn, y_Mo, y_Ni

### Correlation of between wavlength and Cr, Mn, Mo, Ni content

In [ ]:
def correlation(X, y_Cr, y_Mn, y_Mo, y_Ni):
    data_corr_Cr = []
    data_corr_Mn = []
    data_corr_Mo = []
    data_corr_Ni = []
    
    for wavelength in X.columns:
        data_corr_Cr.append(np.corrcoef(y_Cr, X[wavelength])[0,1])
        data_corr_Mn.append(np.corrcoef(y_Mn, X[wavelength])[0,1])
        data_corr_Mo.append(np.corrcoef(y_Mo, X[wavelength])[0,1])
        data_corr_Ni.append(np.corrcoef(y_Ni, X[wavelength])[0,1])
    
    return pd.DataFrame({'Cr': data_corr_Cr, 'Mn': data_corr_Mn, 'Mo': data_corr_Mo, 'Ni': data_corr_Ni}, index = X.columns)

### Filtering lines with higest correlation:  
- select wavelengts for given element where correlation is greater than 0.7 and for other elements less than 0.5

In [ ]:
def filter_lines_with_best_correlation(X, element, cor_threshold, y_Cr, y_Mn, y_Mo, y_Ni):
    data_corr=correlation(X, y_Cr, y_Mn, y_Mo, y_Ni)
    elements = {'Cr', 'Mn', 'Mo', 'Ni'}
    elements.remove(element)
    return X[data_corr.loc[(data_corr[element] >= cor_threshold) & (data_corr[list(elements)[0]] < 0.5) & (data_corr[list(elements)[1]] < 0.5) & (data_corr[list(elements)[2]] < 0.5)].index]    

# TEST - XGBRegressor

### Selecting lines from data with noise spectra filtered out

In [ ]:
def preprocessed_data(X, element='Mn'):
    X_filtered = filter_noise(X = X_training, treshold = 0.05)
    
    X_selected = select_wavelengts_with_big_variance(X_filtered, 500)
    
    y_Cr, y_Mn, y_Mo, y_Ni = update_dependent_variable(X_selected, data_training)
    
    X_filtered_lines = filter_lines_with_best_correlation(X_selected, element, 0.5, y_Cr, y_Mn, y_Mo, y_Ni)
    y = {'Cr': y_Cr, 'Mn': y_Mn, 'Mo': y_Mo, 'Ni': y_Ni}
    
    X_train, X_test, y_train, y_test = train_test_split(X_filtered_lines, y[element], test_size=0.33, random_state=42)
    
    return X_train, X_test, y_train, y_test

In [10]:
def tune_model(X_train, y_train):
    xgb = XGBRegressor()
    
    parameters = {'nthread':[4], #when use hyperthread, xgboost may become slower
              'objective':['reg:squarederror'],
              'learning_rate': [.03, 0.05, .07], #so called `eta` value
              'max_depth': [5, 6, 7],
              'min_child_weight': [4],
              'subsample': [0.7],
              'colsample_bytree': [0.7],
              'n_estimators': [500]}
    
    xgb_grid = GridSearchCV(xgb,
                       parameters,
                        cv = 5,
                        n_jobs = 5,
                        verbose=True)
    
    xgb_grid.fit(X_train, y_train)
    
    print('Best score:', xgb_grid.best_score_)
    print('Best parameters:', xgb_grid.best_params_)
    
    return xgb_grid.best_estimator_

In [ ]:
report_df = pd.DataFrame() 
for element in ['Cr', 'Mn', 'Ni']:
    print(element)
    X_train, X_test, y_train, y_test = preprocessed_data(X_training, element=element)
    xgb_tuned_model = tune_model(X_train, y_train)
    y_pred = xgb_tuned_model.predict(X_test)
    x = np.array(y_test)
    y = np.array(y_pred)
    print()
    r2 = np.round(r2_score(y_test, y_pred),3)
    mae = np.round(mean_absolute_error(y_test, y_pred),3)
    report_df[element] = [r2, mae]
    reg_plot = sns.regplot(x=x,y=y,ci=95);
    reg_plot.set_xlabel('True', fontsize = 20)
    reg_plot.set_ylabel('Prediction', fontsize = 20)
    reg_plot.set_title(element+r' $R^{2}$: '+str(r2)+ ' MAE: '+ str(mae),fontsize = 20)
    reg_plot.grid()
    plt.show();

In [ ]:
report_df['score'] = ['R2','MAE']
report_df.set_index('score')

### PCA 

In [11]:
def preprocessed_data_pca(X, data_training, element='Mn'):
    X_filtered = filter_noise(X = X_training, treshold = 0.05)
    X_avg_limited = limit_wavelength(X_filtered, w_min = 225, w_max = 940)
    X_baseline_corrected = correct_baseline(X_avg_limited)
    y_Cr, y_Mn, y_Mo, y_Ni = update_dependent_variable(X_baseline_corrected, data_training)
    y = {'Cr': y_Cr, 'Mn': y_Mn, 'Mo': y_Mo, 'Ni': y_Ni}
    
    pca = PCA(n_components = 20)
    X_normalized = Normalizer(norm='l2').fit_transform(X_baseline_corrected)
    X_train, X_test, y_train, y_test = train_test_split(X_normalized, y[element], test_size=0.33, random_state=42)
    
    pca.fit(X_train)
    
    projected_train = pca.transform(X_train)
    
    return projected_train, y_train, pca, X_test, y_test

In [ ]:
report_df_pca = pd.DataFrame() 
for element in ['Cr', 'Mn', 'Mo' ,'Ni']:
    print(element)
    
    projected_train, y_train, pca, X_test, y_test = preprocessed_data_pca(X = X_training, data_training = data_training, element = element)
    xgb_tuned_model = tune_model(projected_train, y_train)
    
    projected_test = pca.transform(X_test)
    y_pred = xgb_tuned_model.predict(projected_test)
    x = np.array(y_test)
    y = np.array(y_pred)
    
    print()
    r2 = np.round(r2_score(y_test, y_pred),3)
    mae = np.round(mean_absolute_error(y_test, y_pred),3)
    report_df_pca[element] = [r2, mae]
    reg_plot = sns.regplot(x=x,y=y,ci=95);
    reg_plot.set_xlabel('True', fontsize = 20)
    reg_plot.set_ylabel('Prediction', fontsize = 20)
    reg_plot.set_title(element + r' $R^{2}$: ' + str(r2) + ' MAE: ' + str(mae), fontsize = 20)
    reg_plot.grid()
    plt.show();

### PCA 

In [ ]:
report_df_pca['score'] = ['R2','MAE']
report_df_pca.set_index('score')

### Line selection

In [ ]:
report_df.set_index('score')

## PyCaret